In [1]:
import pandas as pd
import re
import numpy as np
import os
import glob

In [2]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Padronizacao_conversao'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [3]:
regex = {}
regex['ATIVO']                    = {'regex':  "1            ",'executar_variacao': None}
regex['ATIVO_CIRCULANTE']         = {'regex':  "12           ",'executar_variacao': None}
regex['CREDITO_OPER_PS']          = {'regex':  "123          ",'executar_variacao': None}
regex['PASSIVO']                  = {'regex':  "2            ",'executar_variacao': None}
regex['PASSIVO_CIRCULANTE']       = {'regex':  "21           ",'executar_variacao': None}
regex['PASSIVO_NAO_CIRCULANTE']   = {'regex':  "23           ",'executar_variacao': None}
regex['PL']                       = {'regex':  "25           ",'executar_variacao': None}
regex['RECEITA']                  = {'regex':  "31           "}
regex['CONTRAPRESTACAO_PRE']      = {'regex':  "311[^7].1       ", '2012': "311111101    "}
regex['CONTRAPRESTACAO_POS']      = {'regex':  "311[^7].2       ", '2012': "311112101    "}
regex['CORRESP_CEDIDA_PRE']       = {'regex':  "3117.1       "}
regex['CORRESP_CEDIDA_POS']       = {'regex':  "3117.2       "}
regex['DESPESA']                  = {'regex':  "41           "}
regex['EVENTOS_INDENIZAVEIS_PRE'] = {'regex':  "411..1       "}
regex['EVENTOS_INDENIZAVEIS_POS'] = {'regex':  "411..2       "}
regex['VARIACAO_PEONA']           = {'regex':  "414          "}
regex['DESPESA_COM']              = {'regex':  "43           "}
regex['DESPESA_ADM']              = {'regex':  "46           "}
regex['PROVISOES_TECNICAS']       = {'regex' : "2[1|3]1          "}
regex['PPCNG']                    = {'regex' : "2[1|3]1....11    "}
regex['REMISSAO']                 = {'regex' : "2[1|3]1....13    "}
regex['PESL_SUS']                 = {'regex' : "2[1|3]1....2     "}
regex['PESL_OUTROS']              = {'regex' : "2[1|3]1....3     "}
regex['PEONA_OUTROS']             = {'regex' : "2[1|3]1....41    "}
regex['PEONA_SUS']                = {'regex' : "2[1|3]1....42    "}



In [4]:
PASTA_DEMONSTRACOES_POR_ANO = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['pasta_arquivos_contenados']

In [5]:
lista_arquivos = [i for i in glob.glob(PASTA_DEMONSTRACOES_POR_ANO+"/*")]

In [6]:
lista_arquivos

['/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2019todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2013todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2018todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2011todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2012todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2014todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2015todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2016todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2017todas_demonstracoes_concatenadas.csv']

In [7]:
def ordenar_dataset(data):
    return data.sort_values(by=["REG_ANS",'ANO','SEMESTRE',"CD_CONTA_CONTABIL"])

In [8]:
def ajustar_linhas(data):
    data['CD_CONTA_CONTABIL'] = data['CD_CONTA_CONTABIL'].str.ljust(width=13, fillchar=' ')
    data['VL_SALDO_FINAL'] = data['VL_SALDO_FINAL'].str.replace(pat=',', repl='.').astype(float)
    data = ordenar_dataset(data)
    return data

In [9]:
def recuperar_trimestre_anterior(item,data):
    try: 
        chave_anterior = item.name[:item.name.rfind("_")+1]
        semestre_anterior = int(item['SEMESTRE']) - 1
        if (semestre_anterior == 0):
            return None
        chave_anterior = chave_anterior+str(semestre_anterior)
        item_anterior =  data.loc[[chave_anterior]]
        if(len(item_anterior) > 0):
            item_anterior = item_anterior.squeeze()
            return item_anterior
    except:
        return None

In [10]:
def process(item,data,_param_conta):
    valor_atual = item['VL_SALDO_FINAL']
    if(item['SEMESTRE'] != "1"):
        try:
            item_anterior = recuperar_trimestre_anterior(item,data)
            if "executar_variacao" in _param_conta:
                pass
            else:
                valor_atual = valor_atual - item_anterior['VL_SALDO_FINAL']
        except:
            pass
    item[_nome_conta] = str(float("{0:.2f}".format((valor_atual))))
    return item    

In [11]:
def ajustar_contas(_nome_conta, _param_conta, data, lista_codigos):
    _ano = data['ANO'].unique().tolist()[0]
    _regex = _param_conta['regex']
    if _ano in _param_conta:
        _regex = _param_conta[_ano]
    print(_ano,_nome_conta,"-->"+_regex+"<--")
    r = re.compile(_regex)
    lista_match_regex = list(filter(r.match, lista_codigos)) 
    df_contas = data[data['CD_CONTA_CONTABIL'].isin(lista_match_regex)]
    df_contas.insert(loc=0, column='pos', value=np.arange(len(df_contas)))
    df_contas = df_contas.apply(lambda x : process(x,df_contas,_param_conta),axis=1)
    df_contas.drop(columns=['pos'],inplace=True)
    return df_contas

In [12]:
if (not(os.path.exists("resultado"))):
    os.mkdir("resultado")
for _arquivo in lista_arquivos:
    data = pd.read_csv(_arquivo, dtype=str,encoding="ISO-8859-1")
    print(_arquivo)
    data = data.set_index("CHAVE")
    data = ajustar_linhas(data)
    lista_codigos_conta_contabil = data['CD_CONTA_CONTABIL'].unique().tolist()    
    for _nome_conta, _param_conta in regex.items():
        data.insert(loc=3, column=_nome_conta, value=0.0)
        df_contas = ajustar_contas(_nome_conta,_param_conta,data,lista_codigos_conta_contabil)
        data.drop(df_contas.index,inplace=True)
        data = pd.concat([data,df_contas])
    nome_arquivo = _arquivo[_arquivo.rfind(os.path.sep)+1:]
    print(nome_arquivo)
    data = ordenar_dataset(data)
    data.to_csv("resultado"+os.path.sep+nome_arquivo,encoding="ISO-8859-1")

/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2019todas_demonstracoes_concatenadas.csv
2019 ATIVO -->1            <--
2019 ATIVO_CIRCULANTE -->12           <--
2019 CREDITO_OPER_PS -->123          <--
2019 PASSIVO -->2            <--
2019 PASSIVO_CIRCULANTE -->21           <--
2019 PASSIVO_NAO_CIRCULANTE -->23           <--
2019 PL -->25           <--
2019 RECEITA -->31           <--
2019 CONTRAPRESTACAO_PRE -->311[^7].1       <--
2019 CONTRAPRESTACAO_POS -->311[^7].2       <--
2019 CORRESP_CEDIDA_PRE -->3117.1       <--
2019 CORRESP_CEDIDA_POS -->3117.2       <--
2019 DESPESA -->41           <--
2019 EVENTOS_INDENIZAVEIS_PRE -->411..1       <--
2019 EVENTOS_INDENIZAVEIS_POS -->411..2       <--
2019 VARIACAO_PEONA -->414          <--
2019 DESPESA_COM -->43           <--
2019 DESPESA_ADM -->46           <--
2019 PROVISOES_TECNICAS -->2[1|3]1          <--
2019 PPCNG -->2[1|3]1....11    <--
2019 REMISSAO -->2[1|3]1....13    <--
2019 PESL_SUS -->2[1|3]1....2     <--
2019 PESL_OUTROS 

In [13]:
lista_arquivos = [i for i in glob.glob("resultado/*")]

In [14]:
df_demonstracoes_concatenadas = pd.concat([pd.read_csv(f,dtype=str,encoding="ISO-8859-1") for f in lista_arquivos],sort=False)

In [17]:
len(df_demonstracoes_concatenadas)

7580861

In [18]:
df_demonstracoes_concatenadas = df_demonstracoes_concatenadas.groupby(['REG_ANS','ANO','SEMESTRE'],as_index=False).sum()

In [19]:
df_demonstracoes_concatenadas = df_demonstracoes_concatenadas[[
    'REG_ANS', 'ANO', 'SEMESTRE','PEONA_SUS',
       'PEONA_OUTROS', 'PESL_OUTROS', 'PESL_SUS', 'REMISSAO', 'PPCNG',
       'PROVISOES_TECNICAS', 'DESPESA_ADM', 'DESPESA_COM', 'VARIACAO_PEONA',
       'EVENTOS_INDENIZAVEIS_POS', 'EVENTOS_INDENIZAVEIS_PRE', 'DESPESA',
       'CORRESP_CEDIDA_POS', 'CORRESP_CEDIDA_PRE', 'CONTRAPRESTACAO_POS',
       'CONTRAPRESTACAO_PRE', 'RECEITA', 'PL', 'PASSIVO_NAO_CIRCULANTE',
       'PASSIVO_CIRCULANTE', 'PASSIVO', 'CREDITO_OPER_PS', 'ATIVO_CIRCULANTE',
       'ATIVO']]

In [20]:
if (not(os.path.exists('../entrada'))):
    os.mkdir('../entrada')

In [21]:
df_demonstracoes_concatenadas.to_csv("../entrada/todas_demonstracoes_sumarizadas.csv",encoding="ISO-8859-1",index=False)